# Exemplo 18: Hospitalidade - Dashboard de Ocupação (Dask)

Este notebook demonstra o processamento de logs de reservas de hotéis para calcular taxas de ocupação e revenue usando **Dask**.

**Cenário**: Analisar a ocupação de uma cadeia de hotéis para ajustar estratégias de marketing.

## 1. Configuração

In [ ]:
!pip install -q dask[complete] distributed

In [ ]:
from dask.distributed import Client
client = Client()
client

## 2. Gerar Logs de Reservas

In [ ]:
import dask.dataframe as dd
import pandas as pd
import numpy as np
import os

os.makedirs('hotel_data', exist_ok=True)

def generate_bookings(month):
    hotels = [f'HOTEL-{city}' for city in ['NYC', 'LA', 'MIA', 'CHI', 'SF']]
    num_bookings = 2000
    
    df = pd.DataFrame({
        'booking_id': range(num_bookings),
        'hotel_id': np.random.choice(hotels, num_bookings),
        'check_in': pd.date_range(start=f'2024-{month:02d}-01', periods=num_bookings, freq='3min'),
        'nights': np.random.randint(1, 8, num_bookings),
        'price_per_night': np.random.uniform(100, 500, num_bookings),
        'status': np.random.choice(['CONFIRMED', 'CANCELLED'], num_bookings, p=[0.8, 0.2])
    })
    
    df.to_csv(f'hotel_data/month_{month}.csv', index=False)

print("Gerando dados de 12 meses...")
for m in range(1, 13):
    generate_bookings(m)

## 3. Análise de Ocupação e Receita

In [ ]:
ddf = dd.read_csv('hotel_data/month_*.csv')

# Filtrar apenas confirmados
confirmed = ddf[ddf['status'] == 'CONFIRMED']

# Calcular receita total e diária média por hotel
confirmed['total_price'] = confirmed['nights'] * confirmed['price_per_night']

stats = confirmed.groupby('hotel_id').agg(
    total_revenue=('total_price', 'sum'),
    total_bookings=('booking_id', 'count'),
    avg_stay=('nights', 'mean')
)

results = stats.compute()
print("Desempenho por Hotel (Ano Completo):")
print(results.sort_values('total_revenue', ascending=False))

In [ ]:
client.close()